# Solid Ideas

SOLID is a collection of 5 design principles: the Single Responsibility Principle, the Open Closed Principle, the Liskov Substitution Principle, the Interface Segregation Principle and Dependency Inversion.

## Single Responsibility Principle

While the idea that a function or method should have one job is embedded in most engineers consciousnesses, this is the idea that a class or module should, similarly, only have one job.

In [1]:
class StatisticalAnalyzer:
    def analyze(self, statistics):
        self.analysis = {
            'mean': statistics.mean(),
            'max': statistics.max(),
            'min': statistics.min()
        }
        
    def print_analysis(self):
        print(
            'MEAN', self.analysis['mean'],
            'MAX', self.analysis['max'],
            'MIN', self.analysis['min']
        )

The issue here is that we are doing two tasks - analysing data and printing that analysis out. Why is that an issue? Firstly, you can see the tight coupling between printing and analysis - if the printing was handled by a separate class, the interdependency would be clearer, and the responsibility of StatisticalAnalyzer to return a consistent format would be highlighted. As it is, a change to the analysis requires a change to the outputting system, but perhaps this isn't obvious.

A second issue is that, if printing isn't the responsibility of the analyser, then a user can more easily swap output types - why should the output method matter to the statistical tool? Perhaps the user wishes to output JSON, or even HTML.

The final issue, like most design principles, is something more abstract: SRP helps you write maintenable, learnable, clear code, neatly segmented for future developers.

While this is a hard technique to enforce, a few tips to spot an issue are (1) keeping one (or a few intertwined) classes in a module - more modules, fewer classes per module - and (2) checking the imports in each to see if you are pulling from multiple domains. Should this module be using both `pandas` and `requests`?

## Open-Closed Principle

This is a shortening of "Open for Extension, Closed for Modification". This is a way of thinking about how you relate your classes and subclasses. A clearer way, in my opinion, that it has been expressed is:

> You should be able to extend the behavior of a system without having to modify that system.

In [2]:
from smtplib import SMTP

class Mailer:
    def mail(self, email, content):
        with SMTP(self.domain, self.port) as smtp:
            smtp.send_message(content, to_addrs=[email])

This class is a mailer - but it assumes that every emailing is via SMTP... if we wish to use a different approach, or even a different SMTP library, or one we had preconfigured with certain headers not known to the Mailer class, we would have to "open" this file to "modify" it, because it is "closed" to "extension".

In [4]:
class Mailer:
    def mail(self, email, content):
        with self.mailer as smtp:
            smtp.send_message(content, to_addrs=[email])
            
class SmtpMailer(Mailer):
    mailer = SMTP

Not perfect, but this is slightly better - now a subclass can replace the actual mailer doing the action. Why is that useful? Well, aside from sending, e.g. an mail-request to an event queue for some other process to send to SMTP (perhaps to manage rate-limiting across parallel processes), a use-case is a dummy mailer - maybe we just want to test our code without sending emails!

In [10]:
class DummySMTP:
    def __init__(self, domain, port):
        self.domain = domain
        self.port = port
        
    def start(self):
        print('START')
        
    def send_message(self, content, to_addrs):
        print(content, to_addrs)
        
    def finish(self):
        print('END')

This is a handy point to introduce `contextlib` - I mentioned you could create your own `with` statement resources...

In [11]:
import contextlib

@contextlib.contextmanager
def dummy_smtp(domain, port):
    dummy_smtp = DummySMTP(domain, port)
    dummy_smtp.start()
    try:
        yield dummy_smtp
    finally:
        dummy_smtp.end()

The code above is sufficient to let `dummy_smtp` work in a `with` statement. When `with` is called, all the code inside runs up to the `yield`. If an exception (or normal exit) occurs, the `finally` block is run - i.e. in all cases.

This is now enough, that just like our two-line SMTPMailer subclass, we can have a DummyMailer subclass.

In [12]:
class DummyMailer(Mailer):
    mailer = dummy_smtp

Mailer has let us achieve our goal by being open for extension, but we haven't needed to modify the class (i.e. it's closed for modification).

### Exercise: Open a File

Create your own contextmanager that opens a file, yields it and closes the file in a finally block. Try it and make sure it works the same way as `with open('myfile.txt', 'w')`

## Liskov Substitution Principle

The Liskov Substitution Principle is a more conceptual principle than the previous one - anything that uses a class, should be able to use any subclass without knowing (call the same methods, expect the same basic behaviours). A concrete example helps.

In [16]:
class Language:
    def translate(self, text):
        return ...

class Translator:
    def say(self, text: str, language: Language) -> str:
        return language.translate(text)

This seems fine, right?

In [17]:
class SignLanguage(Language):
    def translate(self, text):
        return ... # Well, not a string anyway

So our translator could be given an object of a subclass of Language, SignLanguage, and suddenly its translate method has broken! This is a violation of LSP somewhere. One way to address is to have a subclass `SpokenLanguage` of `Language` and make `Translator` dependent on that instead. Another is to make the translator more flexible on what it will accept back.

Python's duck-typing philosophy really shines here, but is simultaneously the greatest challenge - substitution is much easier because Python discourages unnecessary type-checking (decreasing the likely LSP issues), but we are also more likely to be handed something we aren't expecting.

As in all Python cases, the most generic approach is to make sure you have a reasonable number (but not excessive) try-except statements, and a reasonable fallback. But it doesn't substitute for good planning.

This is a particularly clear cut example, but a common, more abstract one, is a Rectangle. A Square seems like a reasonable subclass of a rectangle, but suppose a rectange has a settable height and width... any function consuming a Rectangle may wish to set the height and or width, but what happens if it's given a Square?

Several options exist to mitigate that issue while protecting LSP, for instance:

* height and width are immutable in a Rectangle - the problem never arises
* changing height or width of a Square changes both - should be clearly documented
* Square isn't made a subclass of Rectangle - if you still want to type-hint for Rectangle and include a Square, Python also lets you use Union types: `Union[Rectangle, Square]`

Another way to violate LSP is to require things in a subclass that anyone using the superclass would not provide. 
A notorious restatement of LSP (echoing Python's duck-typing principle):

> If it walks like a duck, quacks like a duck, but needs batteries, you probably have the wrong abstraction

Perhaps this takes the form of an initialization method that must be called in your subclass (can you override and extend an existing one?), or a particular object that must be passed, not required for instances of the superclass.

How can we help reduce LSP issues? Well, the first is having clear type-hinting in the superclass, so we can see what we should be returning. However, following the Open/Closed Principle, it's then even more important we get our abstraction right so we don't just scope-out SignLanguage by insisting all languages return a `str` for `translate`. Still, lets say we make `SpokenLanguage` our class - how do we help ensure subclasses do what we expect?

In [24]:
class SpokenLanguage:
    def translate(self, text) -> str:
        raise NotImplementedError()
    
class MathematicsTheLanguageOfTheUniverse(SpokenLanguage):
    def translate(self, text) -> int:
        return len(text)
    
with open('test.py', 'w') as f:
    f.write(In[-1])

In [25]:
!mypy test.py

test.py:6: error: Return type "int" of "translate" incompatible with return type "str" in supertype "SpokenLanguage"
test.py:10: error: Name 'In' is not defined
Found 2 errors in 1 file (checked 1 source file)


Also notice that one way we have implemented an abstract method is using the `NotImplementedError` exception. Some static checkers will spot this, but having no compile step, you are then relying on static checking to enforce it manually.

This doesn't really help us in the Rectangle/Square example - there, as in other languages, we simply need to spot that, when creating our Square class, we are introducing constraints that do not exist on the Rectangle superclass.

## Interface Segregation Principle

This is one that turns up less in Python - the best example, across languages, is when an interface (as in Java or C#) requires methods that a perfectly legitimate class implementing it doesn't want to provide.

In [2]:
import abc

class Animal(abc.ABC):
    @abc.abstractmethod
    def walk(self):
        ...
        
class Dolfin(Animal):
    ...
    
Dolfin()

TypeError: Can't instantiate abstract class Dolfin with abstract methods walk

In [7]:
class Animal:
    def walk(self):
        raise NotImplementedError("This animal cannot walk")
        
class Monkey(Animal):
    def walk(self):
        ...
        
class Dolfin(Animal):
    ...
    
Dolfin().walk()

NotImplementedError: This animal cannot walk

So, at this point, we have to question whether `walk` is a reasonable thing to add as a requirement for all Animals. Also notice that we have used `abc`, a Python built-in module for strongly enforcing abstract methods. This isn't as common as it could be, as Python is focused on encouraging try-except in consumer functions, rather than assuming functionality is available up front (the main reason for adding an interface). However, if you are building a system that will be extensible but needs to implement certain subclasses, absolutely essentially, then `abc` is a good way of enforcing that. Unfortunately, the exception will only be raised when the subclass is instantiated, but that's slightly better than waiting until the missing method is called!

In some ways, implementing abstract classes with NotImplementedError in the superclass helps side-step this in Python - subclassing it doesn't force you to implement that method. For optionally-implementable methods, this is a reasonable approach. But you still need to be careful what you consider optional...

## Dependency Inversion

This is one area that is as easy in Python as any other language (although PHP frameworks are notably fantastic at streamlining this). It's pretty simple - remember our Mailer class that wanted an SMTP object?

In [8]:
from smtplib import SMTP

class Mailer:
    def mail(self, email, content):
        with SMTP(self.domain, self.port) as smtp:
            smtp.send_message(content, to_addrs=[email])
            
mailer = Mailer()

Well, why did it need to embed that in its definition? We did one better by subclasses, removing the direct dependency on SMTP to SMTPMailer, _but_ if all we are doing is changing one member, is there a better way?

In [9]:
from smtplib import SMTP

class Mailer:
    def __init__(self, mailer):
        self.mailer = mailer
        
    def mail(self, email, content):
        with self.mailer(self.domain, self.port) as smtp:
            smtp.send_message(content, to_addrs=[email])
            
mailer = Mailer(SMTP)

This is a nicer approach - we no longer assume what the actual mechanism is, we allow the calling class to swap out SMTP for a DummySMTP object (or any other it likes) and it makes Mailer more focused on its original purpose.

A critical benefit of this, to generalize, is test code - you can do BDD and TDD more easily, if you know your Mailer isn't automatically going to start sending things out. All your unit tests (which expect that you aren't chucking other random classes into your tested class) will pass in DummyMailer, or a mock object, and be able to examine what has been done to it.

It also helps the programmer to focus on defining a good SRP-friendly Mailer class, that isn't focused on external assumptions (such as the class it is using), but does its job well.

In other languages, this can encourage a proliferation of interfaces - perhaps using an new `abc` Abstract Base Class as a type-hint to ensure that the `mailer` passed into the constructor is indeed something that can be used inside the `mail` method. However, in Python, this could quickly reduce the succinctness and clarity of the code - and may run a little too far against the duck-typing principle.

Leaving the caller to pass a sensible mailer to the constructor (while clearly documenting expectations of methods, etc.) and, if necessary, using a try-except to provide a fallback approach if `with self.mailer...` doesn't work, may be a more appropriate idea in this context.

----

## Law of Demeter

Lastly, a principle that isn't one of the five SOLID principles, but often comes up in similar conversations, is the Law of Demeter. This is frequently broken in Python - but it's perhaps less of an imperative. That said, it's important to think about. One of my favourite, slightly graphic explanations of this is:

> If a waiter asks a diner for payment, they should wait until that person has taken their wallet out and handed them the money. They should not reach into the diner's pocket, grab the wallet, and take the money out themselves.

In [36]:
class Wallet:
    money = 50
    
    def extract(self, amount):
        self.money -= amount

class Diner:
    def __init__(self):
        self.wallet = Wallet()
    
class Waiter:
    money_on_tray = 0
    
    def request_payment(self, diner, amount):
        diner.wallet.extract(amount)
        self.money_on_tray += amount
        
waiter = Waiter()
diner = Diner()
waiter.request_payment(diner, 10)

...versus...

In [35]:
class Wallet:
    money = 50
    
    def extract(self, amount):
        self.money -= amount

class Diner:
    def __init__(self):
        self.wallet = Wallet()
    
    def give_money(self, amount):
        self.wallet.extract(amount)
        return amount
    
class Waiter:
    money_on_tray = 0
    
    def request_payment(self, diner, amount):
        diner.give_money(amount)
        self.money_on_tray += amount
        
waiter = Waiter()
diner = Diner()
waiter.request_payment(diner, 10)

Why? On a technical level, because how the Diner is managing their money is none of the waiter's business. Should it matter if they have a purse instead? Or they want to use card? Clearly not, but once the waiter class uses `diner.wallet` the assumption is in-built. If `diner` was coming from a third-party class, an upgrade could easily break `waiter` - presumably, the third-party `Diner` class is giving no future guarantees about how it manages its internal properties.

_However_ , the Law of Demeter is much less important in Python - clarity and conciseness encourage some reaching, at the expense of this encapsulation. Therefore, the Law of Demeter should be seen more as an encouragement. As in all languages, the easiest way to spot it, although not foolproof, is a long series of dots:

    thing.method().other_thing.other_method()
    
The more dots, the more brittle your code is likely to be, and susceptible to cause code-base breakages when an apparently unrelated class changes.

**Exception**: one common pattern is to return the same object repeatedly to allow _chaining_. This appears in a number of common Python libraries, such as `pandas` or `SqlAlchemy` (database ORM). In this case, LOD is not violated by a long sequence of dots because _the same object is returned by every call in the chain_. E.g. `df.transpose().apply(fn, axis=1).sum()`

**Note for writing classes**: If you do not wish external users to depend on a property or method in your class, prefix it with an underscore. If you are consuming them, see an underscore prefix as a red-flag - do not use. This is an alternative approach to help address the underlying issues of LOD.

### Exercise: Breaking Solid Ground

Write a piece of code that breaks all six of these, as hard and obviously dangerously as possible.

In [16]:
class Animal:
    name = 'Unknown'
    
    # OCP: Even if our animal doesn't have legs, it will nonetheless have a legs
    # list, if this constructor gets called. If users of the Animal class use this
    # directly, they could get silent bugs by looking at e.g. dolphin.legs
    def __init__(self):
        self.legs = ['left-back', 'right-back', 'left-front', 'right-front']
        
    # ISP: not all animals have legs, why are we requiring them to have leg-names?
    @abc.abstractmethod
    def get_leg_names(self):
        pass

class Giraffe(Animal):
    name = 'Giraffe'
    
    def get_leg_names(self):
        return ', '.join(self.legs)
    
class Tarantula(Animal):
    name = 'Tarantula'
    
    def __init__(self):
        self.legs = [
            'left-front', 'left-middle-front', 'left-middle-back', 'left-back',
            'right-front', 'right-middle-front', 'right-middle-back', 'right-back'
        ]
    
    def get_leg_names(self):
        return ', '.join(self.legs)

class Butterfly(Animal):
    name = 'Butterfly'
    has_transformed = None
    
    # LSP: we are expecting this to initialize whether
    # our butterfly is still a caterpillar or not, but that's
    # not something that users of the superclass will do...
    def set_is_transformed(self, transformed):
        self.has_transformed = transformed
        
    def get_leg_names(self):
        return ['leg {n}' for n in range(6 if self.has_transformed else 16)]

class Zoo:
    def __init__(self):
        # DIP: bit of an extreme example, but rather than saying which animals
        # are in our zoo, or passing a creator function, the constructor will
        # only ever work exactly with these three animals.
        self.animals = [
            Tarantula(),
            Butterfly(),
            Giraffe()
        ]
    
    # SRP: we are baking in how animals are shown to our domain model class
    # (might also notice this links to separation of layers in DDD/MVC)
    def print_animals():
        for animal in self.animals:
            print(f'A {animal.name} has {len(animal.get_leg_names())}')
            
zoo = Zoo()

# LOD: not only are we baking in the assumption that animals have legs,
# we're baking in the way that's stored - note that this has gone wrong for
# Butterflies who do not use the legs list
sum((zoo.animals[i].legs for i in range(3)), [])

['left-front',
 'left-middle-front',
 'left-middle-back',
 'left-back',
 'right-front',
 'right-middle-front',
 'right-middle-back',
 'right-back',
 'left-back',
 'right-back',
 'left-front',
 'right-front',
 'left-back',
 'right-back',
 'left-front',
 'right-front']